In [1]:
import pymysql
import pandas as pd
import folium
from branca.colormap import linear
from datetime import datetime, timedelta
from connections import DATASET_DATABASE

## Traccar DB에서 GPS 위치 정보 가져오기

In [2]:
def conn_raw_mysql() -> pymysql.connect:
    """
    Raw DB의 MySQL Connection 정보를 세팅하여 반환한다
    
    :return: pymysql.connect
    """
    host = DATASET_DATABASE['HOST']
    user = DATASET_DATABASE['USER']
    password = DATASET_DATABASE['PASSWORD']
    database = DATASET_DATABASE['DATABASE']
    encoding = DATASET_DATABASE['ENCODING']

    conn = pymysql.connect(host=host,
                           user=user,
                           password=password,
                           db=database,
                           charset=encoding)
    return conn



In [3]:
query = """
SELECT deviceid,
       devicetime,
       latitude,
       longitude,
       speed,
       accuracy
FROM tc_positions
LEFT JOIN tc_devices td on td.id = tc_positions.deviceid
WHERE servertime between '2021-11-15 00:00:00' and '2021-11-15 23:59:59';
"""

sql_conn = conn_raw_mysql()
df = pd.read_sql_query(query, sql_conn)
sql_conn.close()


In [4]:
df.head(3)

,deviceid,devicetime,latitude,longitude,speed,accuracy
0,1,2021-11-15 00:19:09,37.630257,126.833032,0.0,1414.000000
1,1,2021-11-15 00:41:15,37.630598,126.830443,0.0,14.255316
2,1,2021-11-15 00:43:04,37.630598,126.830443,0.0,2244.031816


## 위치 정보 전처리

### devicetime UTC -> KST 변환

In [5]:
df['devicetime'] = df['devicetime'].apply(lambda x: x + timedelta(hours=9))

### Speed 단위 변환(knots -> km/h)
 - Traccar에 저장된 speed 유닛의 기본은 단위는 knots이다

In [6]:
df['speed'] = df['speed'].apply(lambda x : x * 1.852)

### 이동 경로에 따라 dataframe 나누기
 - 아침, 저녁, 야간별로 이동 경로가 다르므로 시간을 기준으로 데이터를 자른다
 
 - 첫번째 경로: 고양시 -> 분당(야탑)
 - 두번째 경로: 분당(야탑) -> 숭실대
 - 세번째 경로: 숭실대 -> 고양시

In [7]:
# 데이터를 자르기 위해 데이터간의 시간차를 구한다
# 시간차가 1시간 이상인 경우, 별도의 경로로 구분하여 자른다
df['diffTime'] = df['devicetime'].diff()
df['diffTime'] = df['diffTime'].fillna(pd.Timedelta('00:00:00'))

In [8]:
# 1시간 이상 차이가 나는 데이터를 구한다
split_time_df = df['diffTime'] > pd.Timedelta(1, unit='h')

In [9]:
# 1시간 이상 차이가나는 데이터의 인덱스를 가져온디
split_idx = list(df[split_time_df].index)
split_idx

[72, 156]

In [10]:
# 첫번째 이동 경로의 Dataframe을 필터링한다
route1 = df[:split_idx[0]]

In [11]:
# 두번째 이동 경로의 Dataframe을 필터링한다
route2 = df[split_idx[0]+1:split_idx[1]]

In [12]:
# 세번째 이동 경로의 Dataframe을 필터링한다
route3 = df[split_idx[1]+1:]

## 이동경로 그리기

In [13]:
def draw_location_to_map(loc_df: pd.DataFrame):
    """
    이동경로가 담긴 Dataframe으로 이동경로를 그려 객체를 반환한다
    
    :param df:
    :return:
    """
    # latitude, longitude를 리스트로 변경한다
    locations = [[loc_df.loc[i, 'latitude'], loc_df.loc[i, 'longitude']] for i in loc_df.index]

    # 첫번째 위치를 지도에 표시한다
    route_map = folium.Map(locations[0], tiles='Cartodb Positron', zoom_start=12)
    
    # 차량의 속도를 컬러로 표시하기 위한 컬러 맵을 생성한다
    colormap = linear.RdYlGn_11.scale(float(loc_df['speed'].min()), float(loc_df['speed'].max())).to_step(10)
    colormap.caption = 'Speed'
    colormap.add_to(route_map)

    # 차량의 속도를 컬러로 표시하기 위해, speed 컬럼을 리스트로 생성한다
    speeds = loc_df['speed'].astype('float').to_list()
    
    # 위치 정보를 기준으로 지도에 이동경로를 그린다
    nope = folium.ColorLine(locations, colors=speeds, colormap=colormap, weight=4).add_to(route_map)
    
    return route_map

### 첫번째 경로 : 고양시 -> 분당

In [14]:
map1 = draw_location_to_map(route1)

In [15]:
map1

서부간선지하차도와 제2경인고속도로의 터널내에서 GPS 정보가 잘 수집되지 않으 속도와 위치정보가 일부 정확하게 표시되지 않지만, 경로는 잘 그려진다

----

### 두번째 경로 : 분당 -> 숭실대

In [16]:
map2 = draw_location_to_map(route2)

In [17]:
map2

강남순환로 터널내에서 GPS 정보가 잘 수집되지 않아 속도와 위치정보가 일부 정확하게 표시되지 않지만, 경로는 잘 그려진다

---

In [18]:
map3 = draw_location_to_map(route3)

In [19]:
map3

경로상에 터널이 존재하지 않아 출발지부터 목적지까지 정확하게 경로가 잘 그려진다